In [4]:
suppressWarnings(suppressMessages(library(tidyverse)))
suppressWarnings(suppressMessages(require(reshape)))
suppressWarnings(suppressMessages(library(fdrtool)))
 normalize <- function(x){
  return((x-mean(x,na.rm = TRUE)) / (sd(x,na.rm = TRUE)))
}

In [5]:
filter_genes <- read.table("../../reports/Master Table/Unique_Counts_master_table_Median_long_v2.tab",header=TRUE)

In [6]:
head(filter_genes,2)

,strain,condition,slevel,locus_tag,Median,controlMedian,effect,log2_coefficient,mean,controlmean,fdr,Annotation
,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
1,DCLPA,canavanine,HIGH,CCNA_00001,0,0.0,strainDCLPA:conditionCAN:slevelHIGH,-1.17219507,0.3333333,42.75,4.234444e-06,Essential
2,DCLPA,canavanine,HIGH,CCNA_00002,42,50.5,strainDCLPA:conditionCAN:slevelHIGH,-0.06495413,46.0000000,109.25,1.000000e+00,Conditionally_Neutral


In [7]:
 f_genes <- filter_genes%>%filter(Annotation=="Conditionally_Essential"|Annotation== "Essential")%>%select(strain,condition,slevel,locus_tag)%>%mutate(nonmodel="Yes")

In [5]:
##

In [8]:
TC <- read.table("../../reports/HORSHOE_LFSR/GLMNET_UC_RESULTS.TAB",header=TRUE,check.names=FALSE,fill=TRUE)
TC <- melt(TC,id="locus_tag")
colnames(TC) <- c("locus_tag","Effect","Estimate")

In [9]:
head(TC,2)

,locus_tag,Effect,Estimate
,<chr>,<fct>,<dbl>
1,CCNA_00001,Intercept,3.356621
2,CCNA_00002,Intercept,4.665810


In [10]:
#%>%filter(condition!="NA" & slevel!="NA")

In [11]:
TC_filter <- TC%>%separate(Effect, c("strain","condition","slevel"), ":",remove=FALSE)%>%filter(condition!="NA" & slevel!="NA")%>%mutate(strain=gsub("strain","",strain),
                                                                                                                                   condition=gsub("condition","",condition),
                                                                                                                                   slevel=gsub("slevel","",slevel))%>%
filter(Effect!="Intercept")%>%mutate(condition=gsub("CONTROL","none",condition))

Warning message:
"Expected 3 pieces. Missing pieces filled with `NA` in 81680 rows [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, ...]."


In [12]:
head(TC_filter)

,locus_tag,Effect,strain,condition,slevel,Estimate
,<chr>,<fct>,<chr>,<chr>,<chr>,<dbl>
1,CCNA_00001,strainwild-type:conditionheat:slevelLOW,wild-type,heat,LOW,0.09104253
2,CCNA_00002,strainwild-type:conditionheat:slevelLOW,wild-type,heat,LOW,-0.09296463
3,CCNA_00003,strainwild-type:conditionheat:slevelLOW,wild-type,heat,LOW,-0.24017590
4,CCNA_00004,strainwild-type:conditionheat:slevelLOW,wild-type,heat,LOW,-0.39609785
5,CCNA_00005,strainwild-type:conditionheat:slevelLOW,wild-type,heat,LOW,-0.20476829
6,CCNA_00006,strainwild-type:conditionheat:slevelLOW,wild-type,heat,LOW,-0.11481813


In [11]:
#TC_filter <- TC%>%separate(Effect, c("strain","condition","slevel"), ":",remove=FALSE)%>%drop_na()%>%filter(condition!="conditionnone")%>%mutate(strain=gsub("strain","",strain),
                                                                                                                                 #  condition=gsub("condition","",condition),
                                                                                                                                 #  slevel=gsub("slevel","",slevel))

In [13]:
nrow(f_genes)

[1] 48007

In [14]:
nrow(TC_filter)

[1] 183780

In [15]:
TC <- merge(TC_filter,f_genes,by=c("strain","condition","slevel","locus_tag"),all.x = TRUE)%>%filter(locus_tag!="NA")%>%filter(is.na(nonmodel))%>%select(-nonmodel)

In [16]:
nrow(TC)

[1] 140224

In [17]:
LFDR <- TC%>%group_by(Effect)%>%mutate(coeff_log2value=as.numeric(Estimate)/log(2))%>%mutate(Norm_effect=normalize(coeff_log2value))%>%drop_na(Norm_effect)%>%mutate(fdr=(fdrtool(Norm_effect,cutoff.method="locfdr",plot=FALSE,verbose=FALSE)$lfdr))

In [18]:
TC_results <- merge(filter_genes,LFDR,by=c("locus_tag","strain","condition","slevel"),all.x=TRUE)%>%mutate( Annotation = case_when(
                              fdr.y<0.10 & coeff_log2value >0 ~ "Conditionally_Detrimental",
                              fdr.y<0.10 & coeff_log2value <0 ~ "Conditionally_Beneficial",fdr.y>=0.10~"Conditionally_Neutral",TRUE ~ as.character(Annotation)))%>%select(locus_tag,strain,condition,
                              slevel,Annotation,Median,controlMedian,fdr.y,coeff_log2value,fdr.y)%>%dplyr::rename(fdr=fdr.y)%>%filter(condition!="none")

In [19]:
write.table(TC_results,"../../reports/Master Table/Unique_Counts_FDR_v2.TAB",quote=FALSE,sep="\t")